In [ ]:
pip install -q PyYaml==5.3.1

     |████████████████████████████████| 276kB 4.9MB/s 


In [ ]:
pip install -q rnnmorph==0.4.0

     |████████████████████████████████| 10.5MB 74.3MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 7.1MB 46.8MB/s 


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
DATA_URL = 'http://az.lib.ru/n/nekrasow_n_a/text_1840_pevitza.shtml'

In [ ]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
# Текс с html тегами
raw_text = resource.read().decode(resource.headers.get_content_charset())

In [ ]:
import re

clean_pattern = re.compile("<.*?>")

def clean_html(raw_html: str):
  clean_text = re.sub(clean_pattern, " ", raw_html)
  return clean_text

cleaned_text = clean_html(raw_text)  # Текс без html тегов

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


"A total of 914 'sentences'"

In [ ]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent if str(pred.normal_form).isalpha()] 
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences")]

sentences: 100%|██████████| 914/914 [00:00<00:00, 99514.42it/s]


In [ ]:
len(predictions)

914

In [ ]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens)

12710

In [ ]:
len(list(set(list(tok for tok in non_uniq_tokens if non_uniq_tokens.count(tok) >= 100))))

14

In [ ]:
from operator import itemgetter

In [ ]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
fr_items = FreqDist(non_uniq_tokens)
fr_lst = list([word, freq] for word, freq in fr_items.items())
fr_lst.sort(key=itemgetter(1), reverse=True)
fr_lst2 = list(word for word, freq in fr_lst[0:50])

In [ ]:
fr_lst3 = list(word for word in fr_lst2 if word not in STOPWORDS)
len(fr_lst3)/len(fr_lst2)

0.44